In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import statsmodels.formula.api as smf

from data import load_pcd_df

pio.templates.default = "plotly_white"

In [2]:
api_price_df = pd.read_csv('data/API prices - full view.csv')

api_price_df.head()

,Price description,Price,Price Unit,Model,Fine Tuned Model,Price date,Model Version,Organization (model developer),Organization (API vendor),Context Window,Archived price link,Tags,Notes,Last Modified
0,$2.50 / 1M input tokens,$2.50000,$/1M input tokens,GPT-4o,NaN,2024-08-12,gpt-4o-2024-08-06,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,OpenAI: 50% off for batch submission,NaN,12/7/2024 9:13pm
1,$10.00 / 1M output tokens,$10.00000,$/1M output tokens,GPT-4o,NaN,2024-08-12,gpt-4o-2024-08-06,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,OpenAI: 50% off for batch submission,NaN,12/7/2024 9:13pm
2,$0.638 / 1k 512^2 px input images,$0.63800,$/1k 512^2 px input images,GPT-4o,NaN,2024-08-12,gpt-4o-2024-08-06,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,NaN,High-resolution input images are priced as fol...,12/7/2024 9:13pm
3,$1.275 / 1k 512^2 px input images,$1.27500,$/1k 512^2 px input images,GPT-4o,NaN,2024-08-12,gpt-4o-2024-05-13,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,NaN,High-resolution input images are priced as fol...,12/7/2024 9:13pm
4,$0.213 / 1k low resolution input images,$0.21300,$/1k low resolution input images,GPT-4o,NaN,2024-08-12,gpt-4o-2024-08-06,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,NaN,Low-resolution images use 85 input tokens/imag...,12/7/2024 9:13pm


In [3]:
api_price_df['Price Unit'].unique()

array(['$/1M input tokens', '$/1M output tokens',
       '$/1k 512^2 px input images', '$/1k low resolution input images',
       '$/1k 1024^2 px input images',
       '$/1M input tokens (for <=128k tokens)',
       '$/1M input tokens (for > 128k tokens)',
       '$/1M output tokens (for <= 128k tokens)',
       '$/1M output tokens (for > 128k tokens)',
       '$/1M cashed tokens (for <= 128k tokens)',
       '$/1M cashed tokens (for >128k tokens)',
       '$/1M cashed tokens per hour', '$/1M embedding tokens',
       '$/1M training tokens', '$/1k 1024^2 px images generated',
       '$/minute', '$/1M characters', '$/month'], dtype=object)

In [4]:
# Group by price unit and count
api_price_df.groupby('Price Unit').size()


Price Unit
$/1M cashed tokens (for <= 128k tokens)     3
$/1M cashed tokens (for >128k tokens)       3
$/1M cashed tokens per hour                 2
$/1M characters                             2
$/1M embedding tokens                       3
$/1M input tokens                          46
$/1M input tokens (for <=128k tokens)       3
$/1M input tokens (for > 128k tokens)       3
$/1M output tokens                         30
$/1M output tokens (for <= 128k tokens)     3
$/1M output tokens (for > 128k tokens)      3
$/1M training tokens                        5
$/1k 1024^2 px images generated             8
$/1k 1024^2 px input images                 3
$/1k 512^2 px input images                  3
$/1k low resolution input images            3
$/minute                                    1
$/month                                     1
dtype: int64

In [5]:
# Focus on the simplest price units: $/1M input tokens and $/1M output tokens
api_price_df = api_price_df[api_price_df['Price Unit'].isin(['$/1M input tokens', '$/1M output tokens'])]
api_price_df.head()

,Price description,Price,Price Unit,Model,Fine Tuned Model,Price date,Model Version,Organization (model developer),Organization (API vendor),Context Window,Archived price link,Tags,Notes,Last Modified
0,$2.50 / 1M input tokens,$2.50000,$/1M input tokens,GPT-4o,NaN,2024-08-12,gpt-4o-2024-08-06,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,OpenAI: 50% off for batch submission,NaN,12/7/2024 9:13pm
1,$10.00 / 1M output tokens,$10.00000,$/1M output tokens,GPT-4o,NaN,2024-08-12,gpt-4o-2024-08-06,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,OpenAI: 50% off for batch submission,NaN,12/7/2024 9:13pm
7,$5.00 / 1M input tokens,$5.00000,$/1M input tokens,GPT-4o,NaN,2024-08-12,gpt-4o-2024-05-13,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,OpenAI: 50% off for batch submission,NaN,12/7/2024 9:13pm
8,$15.00 / 1M output tokens,$15.00000,$/1M output tokens,GPT-4o,NaN,2024-08-12,gpt-4o-2024-05-13,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,OpenAI: 50% off for batch submission,NaN,12/7/2024 9:13pm
10,$0.15 / 1M input tokens,$0.15000,$/1M input tokens,GPT-4o mini,NaN,2024-08-12,gpt-4o-mini-2024-07-18,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,OpenAI: 50% off for batch submission,NaN,8/16/2024 2:26pm


In [6]:
# Convert Price string to float
api_price_df['Price (USD)'] = api_price_df['Price'].str.replace('$', '').astype(float)

In [7]:
# Plot $/1M tokens over time
fig = px.scatter(api_price_df, x='Price date', y='Price (USD)', color='Price Unit', title='API Prices Over Time')
# Log y
fig.update_layout(yaxis_type='log')
fig.show()


In [8]:
# Regression on price over time
api_price_df['date'] = api_price_df['Price date'].map(lambda x: pd.Timestamp(x).toordinal())
api_price_df['price'] = api_price_df['Price (USD)']
api_price_df['log_price'] = np.log10(api_price_df['Price (USD)'])
model = smf.ols('log_price ~ date', data=api_price_df[api_price_df['Price Unit'] == '$/1M input tokens']).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_price   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9247
Date:                Thu, 23 Jan 2025   Prob (F-statistic):              0.341
Time:                        11:31:50   Log-Likelihood:                -45.549
No. Observations:                  46   AIC:                             95.10
Df Residuals:                      44   BIC:                             98.76
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    996.7351   1036.412      0.962      0.341   -1092.016    3085.486
date          -0.0013      0.001     -0.962      0.341      -0.004       0.001
==============================================================================
Omnibus:                        2.688   Durbin-Watson:                   1.909
Prob(Omnibus):                  0.261   Jarque-Bera (JB):                2.407
Skew:                          -0.468   Prob(JB):                        0.300
Kurtosis:                       2.384   Cond. No.                     7.80e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.8e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
model = smf.ols('log_price ~ date', data=api_price_df[api_price_df['Price Unit'] == '$/1M output tokens']).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_price   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.016
Method:                 Least Squares   F-statistic:                    0.5359
Date:                Thu, 23 Jan 2025   Prob (F-statistic):              0.470
Time:                        11:31:50   Log-Likelihood:                -33.360
No. Observations:                  30   AIC:                             70.72
Df Residuals:                      28   BIC:                             73.52
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    906.1941   1237.141      0.732      0.470   -1627.974    3440.362
date          -0.0012      0.002     -0.732      0.470      -0.005       0.002
==============================================================================
Omnibus:                        1.228   Durbin-Watson:                   1.897
Prob(Omnibus):                  0.541   Jarque-Bera (JB):                0.809
Skew:                          -0.401   Prob(JB):                        0.667
Kurtosis:                       2.928   Cond. No.                     6.58e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.58e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
# Model Version
api_price_df['Model Version'].unique()

array(['gpt-4o-2024-08-06', 'gpt-4o-2024-05-13', 'gpt-4o-mini-2024-07-18',
       nan, 'Llama-3.1-405B-Instruct', 'o1-preview-2024-09-12',
       'o1-mini-2024-09-12', 'Llama 3 8B 8k', 'Llama 3.1 8B Instruct',
       'Llama 3.1 405B Turbo', 'Llama 3.1 70B Instruct',
       'Jamba 1.5 Large', 'Cygnet'], dtype=object)

In [11]:
# Do a line plot of each Model's price over time
for model in api_price_df['Model'].unique():
    model_df = api_price_df[api_price_df['Model'] == model]
    fig = px.line(model_df, x='Price date', y='Price (USD)', color='Price Unit', 
                  title=f'{model} Prices Over Time', markers=True)
    fig.update_layout(yaxis_type='log')
    fig.show()

# Price vs training compute

In [12]:
pcd_df = load_pcd_df()
pcd_df.head()

,Model,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Hardware type,Training compute estimation method,Biological model safeguards,Hardware utilization (temp),BenchmarkHub-v1,Hugging Face developer id,Post-training compute (FLOP),Post-training compute notes,Hardware maker,benchmarks/models
0,INTELLECT-MATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTELLECT-MATH
1,Cosmos-1.0-\nDiffusion-14B Video2World,"Robotics,Vision,Video","Robotic manipulation,Self-driving car,Video ge...","NVIDIA: Niket Agarwal, Arslan Ali, Maciej Bala...",NaN,NaN,Open weights (restricted use),https://arxiv.org/abs/2501.03575,NaN,Cosmos World Foundation Model Platform for Phy...,...,NaN,Hardware,NaN,NaN,NaN,nvidia,NaN,NaN,NVIDIA,NaN
2,OLMo 2 Furious 7B,Language,"Language modelling/generation,Question answering","Team OLMo, Pete Walsh, Luca Soldaini, Dirk Gro...",NaN,NaN,Open weights (unrestricted),https://arxiv.org/abs/2501.00656,NaN,2 OLMo 2 Furious,...,NaN,"Reported,Operation counting",NaN,NaN,NaN,allenai,NaN,NaN,NVIDIA,NaN
3,OLMo 2 Furious 13B,Language,"Language modelling/generation,Question answering","Team OLMo, Pete Walsh, Luca Soldaini, Dirk Gro...",NaN,NaN,Open weights (unrestricted),https://arxiv.org/abs/2501.00656,NaN,2 OLMo 2 Furious,...,NaN,"Reported,Operation counting",NaN,NaN,NaN,allenai,NaN,NaN,NVIDIA,NaN
4,DeepSeek-V3,Language,"Language modelling/generation,Code generation,...",NaN,Training cost,training cost was $5.3million USD (Table 1),Open weights (restricted use),https://github.com/deepseek-ai/DeepSeek-V3/blo...,NaN,DeepSeek-V3 Technical Report,...,NaN,Operation counting,NaN,NaN,NaN,deepseek-ai,NaN,NaN,NVIDIA,DeepSeek-V3


In [13]:
# Add 'Training compute (FLOP)' column to price_df
# Need to match on 'Model' which is a column in both dataframes
# Use a left join
price_df_cols = api_price_df.columns.tolist()
price_df = api_price_df.merge(pcd_df, on='Model', how='left')
# Drop all PCD columns except 'Training compute (FLOP)'
pcd_cols = [col for col in pcd_df.columns if col not in ['Model', 'Training compute (FLOP)']]
price_df = price_df.drop(columns=pcd_cols)
price_df[price_df['Training compute (FLOP)'].notna()]

,Price description,Price,Price Unit,Model,Fine Tuned Model,Price date,Model Version,Organization (model developer),Organization (API vendor),Context Window,Archived price link,Tags,Notes,Last Modified,Price (USD),date,price,log_price,Training compute (FLOP)
0,$2.50 / 1M input tokens,$2.50000,$/1M input tokens,GPT-4o,NaN,2024-08-12,gpt-4o-2024-08-06,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,OpenAI: 50% off for batch submission,NaN,12/7/2024 9:13pm,2.500,739110,2.500,0.397940,3.810001e+25
1,$10.00 / 1M output tokens,$10.00000,$/1M output tokens,GPT-4o,NaN,2024-08-12,gpt-4o-2024-08-06,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,OpenAI: 50% off for batch submission,NaN,12/7/2024 9:13pm,10.000,739110,10.000,1.000000,3.810001e+25
2,$5.00 / 1M input tokens,$5.00000,$/1M input tokens,GPT-4o,NaN,2024-08-12,gpt-4o-2024-05-13,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,OpenAI: 50% off for batch submission,NaN,12/7/2024 9:13pm,5.000,739110,5.000,0.698970,3.810001e+25
3,$15.00 / 1M output tokens,$15.00000,$/1M output tokens,GPT-4o,NaN,2024-08-12,gpt-4o-2024-05-13,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,OpenAI: 50% off for batch submission,NaN,12/7/2024 9:13pm,15.000,739110,15.000,1.176091,3.810001e+25
4,$0.15 / 1M input tokens,$0.15000,$/1M input tokens,GPT-4o mini,NaN,2024-08-12,gpt-4o-mini-2024-07-18,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,OpenAI: 50% off for batch submission,NaN,8/16/2024 2:26pm,0.150,739110,0.150,-0.823909,7.360010e+24
5,$0.60 / 1M output tokens,$0.60000,$/1M output tokens,GPT-4o mini,NaN,2024-08-12,gpt-4o-mini-2024-07-18,OpenAI,OpenAI,NaN,https://web.archive.org/web/20240812003133/htt...,OpenAI: 50% off for batch submission,NaN,8/16/2024 2:29pm,0.600,739110,0.600,-0.221849,7.360010e+24
6,$15 / 1M input tokens,$15.00000,$/1M input tokens,Claude 3 Opus,NaN,2024-08-12,NaN,Anthropic,Anthropic,NaN,https://archive.is/5C8WA,NaN,NaN,8/13/2024 1:37pm,15.000,739110,15.000,1.176091,1.640001e+25
7,$75 / 1M output tokens,$75.00000,$/1M output tokens,Claude 3 Opus,NaN,2024-08-12,NaN,Anthropic,Anthropic,NaN,https://archive.is/5C8WA,NaN,NaN,8/13/2024 1:37pm,75.000,739110,75.000,1.875061,1.640001e+25
10,$3 / 1M input tokens,$3.00000,$/1M input tokens,Claude 3.5 Sonnet,NaN,2024-08-12,NaN,Anthropic,Anthropic,NaN,https://archive.is/5C8WA,NaN,NaN,12/7/2024 6:47pm,3.000,739110,3.000,0.477121,4.980001e+25
11,$15 / 1M output tokens,$15.00000,$/1M output tokens,Claude 3.5 Sonnet,NaN,2024-08-12,NaN,Anthropic,Anthropic,NaN,https://archive.is/5C8WA,NaN,NaN,12/7/2024 6:47pm,15.000,739110,15.000,1.176091,4.980001e+25


In [14]:
# Use graph_objects to plot price vs training compute
fig = go.Figure()

input_price_df = price_df[price_df['Price Unit'] == '$/1M input tokens'].copy()
output_price_df = price_df[price_df['Price Unit'] == '$/1M output tokens'].copy()

fig.add_trace(go.Scatter(
    x=input_price_df['Training compute (FLOP)'], 
    y=input_price_df['Price (USD)'],
    mode='markers',
    marker=dict(size=10, opacity=0.7)
))
fig.update_layout(
    title='Input Price vs Training Compute',
    xaxis_title='Training Compute (FLOP)',
    yaxis_title='Input Price (USD)',
    yaxis_type='log',
    xaxis_type='log',
)
fig.show()

# Output price vs training compute
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=output_price_df['Training compute (FLOP)'], 
    y=output_price_df['Price (USD)'],
    mode='markers',
    marker=dict(size=10, opacity=0.7)
))
fig.update_layout(
    title='Output Price vs Training Compute',
    xaxis_title='Training Compute (FLOP)',
    yaxis_title='Output Price (USD)',
    yaxis_type='log',
    xaxis_type='log',
)
fig.show()

In [15]:
# Regression on price vs training compute
input_price_df['log_flop'] = np.log10(input_price_df['Training compute (FLOP)'])
input_price_df['price'] = input_price_df['Price (USD)']
model = smf.ols('log_price ~ log_flop', data=input_price_df).fit()
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_price   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.130
Method:                 Least Squares   F-statistic:                     4.598
Date:                Thu, 23 Jan 2025   Prob (F-statistic):             0.0428
Time:                        11:31:50   Log-Likelihood:                -23.592
No. Observations:                  25   AIC:                             51.18
Df Residuals:                      23   BIC:                             53.62
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -8.1205      3.849     -2.110      0.046     -16.083      -0.158
log_flop       0.3303      0.154      2.144      0.043       0.012       0.649
==============================================================================
Omnibus:                        0.590   Durbin-Watson:                   2.275
Prob(Omnibus):                  0.745   Jarque-Bera (JB):                0.049
Skew:                          -0.073   Prob(JB):                        0.976
Kurtosis:                       3.158   Cond. No.                         743.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
# Plot the regression line
compute_range = np.linspace(input_price_df['log_flop'].min(), input_price_df['log_flop'].max(), 100)
predict_df = pd.DataFrame({'log_flop': compute_range})
predict_df['log_price'] = model.predict(predict_df)
fig = px.scatter(input_price_df, x='log_flop', y='log_price', title='Input Price vs Training Compute')
fig.add_trace(go.Scatter(x=predict_df['log_flop'], y=predict_df['log_price'], mode='lines', name='Regression Line'))
fig.show()


In [17]:
output_price_df.loc[:,'log_flop'] = np.log10(output_price_df['Training compute (FLOP)'])
output_price_df.loc[:,'price'] = output_price_df['Price (USD)']
model = smf.ols('log_price ~ log_flop', data=output_price_df).fit()
model.summary()

/Users/bencottier/miniconda3/envs/learning-curves/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning:

kurtosistest only valid for n>=20 ... continuing anyway, n=12



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_price   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     1.276
Date:                Thu, 23 Jan 2025   Prob (F-statistic):              0.285
Time:                        11:31:50   Log-Likelihood:                -11.068
No. Observations:                  12   AIC:                             26.14
Df Residuals:                      10   BIC:                             27.11
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.4577      4.570     -0.975      0.352     -14.641       5.726
log_flop       0.2075      0.184      1.130      0.285      -0.202       0.617
==============================================================================
Omnibus:                        1.187   Durbin-Watson:                   2.544
Prob(Omnibus):                  0.553   Jarque-Bera (JB):                0.781
Skew:                           0.248   Prob(JB):                        0.677
Kurtosis:                       1.853   Cond. No.                         592.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
# Plot the regression line
compute_range = np.linspace(output_price_df['log_flop'].min(), output_price_df['log_flop'].max(), 100)
predict_df = pd.DataFrame({'log_flop': compute_range})
predict_df['log_price'] = model.predict(predict_df)
fig = px.scatter(output_price_df, x='log_flop', y='log_price', title='Output Price vs Training Compute')
fig.add_trace(go.Scatter(x=predict_df['log_flop'], y=predict_df['log_price'], mode='lines', name='Regression Line'))
fig.show()


# Price vs. Inference compute